# Sentiment Analysis - Model Development

## On Regular Feature-Engineered Data

### 1. CV+TF-IDF

In [30]:
import pandas as pd

# reading text-processed data
df=pd.read_csv("https://raw.githubusercontent.com/Alex-Mak-MCW/SpotifyDataScienceProject/main/Data/Sentiment_Analysis/CV_and_TFIDF.csv", encoding='utf-8')

In [31]:
print(df.shape) # (1621, 13104)
df.head()

(1621, 13104)


,track_name,artist,album,release_date,duration,popularity,explicit,lyrics,danceability,energy,...,zoo,zoom,zoomin,zoowap,ztrip,zucchini,zulema,zuli,zz,zzzzzzz
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,200040,90,0,yeah tryna call long enough mayb show love may...,0.514,0.730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,233712,86,0,club best place find lover bar go mm friend ta...,0.825,0.652,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,182160,89,0,go time fear one save noth realli got way driv...,0.501,0.405,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,Hollywood's Bleeding,2019-09-06,157560,85,0,ayi ayi ayi ayi ooh ooh ooh ooh ooh ooh ayi ay...,0.755,0.522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,As It Was,Harry Styles,Harry's House,2022-05-20,167303,91,0,come harri want say goodnight holdin back grav...,0.520,0.731,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Assign x and y assigning

# assign y as target variable
y=df.loc[:, df.columns =='mood'].values.ravel()

# Drop target variable in df
df.drop(['valence','mood', 'lyrics'], axis=1, inplace=True)

# Rest of the variables in df
x=df.iloc[:, 4:]

In [33]:
# data splitting (with options)
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test, y_train, y_test=train_test_split(x,y,test_size=0.2)

# split the 20% by half (10% test, 10% validation)
X_val, X_test, y_val, y_test=train_test_split(X_test,y_test,test_size=0.5)

In [34]:
# verification

print("The shapes in the training dataset:")
print("X: ",X_train.shape)  # (1296, 13097)
print("Y: ",y_train.shape)  # (1296,)

print("The shapes in the valdiation dataset:")
print("X: ",X_val.shape)    # (162, 13097)
print("Y: ",y_val.shape)    # (162,)

print("The shapes in the testing dataset:")
print("X: ",X_test.shape)   # (163, 13097)
print("Y: ",y_test.shape)   # (163,)

The shapes in the training dataset:
X:  (1296, 13097)
Y:  (1296,)
The shapes in the valdiation dataset:
X:  (162, 13097)
Y:  (162,)
The shapes in the testing dataset:
X:  (163, 13097)
Y:  (163,)


#### 1.1 Linear Regression Model

In [35]:
# Model 1: basic LR (<5s)

# init and training the model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize and train the model with L1 regularization (Lasso)
LR = LogisticRegression()  # C is the inverse of regularization strength
LR.fit(X_train, y_train)

# Predict on the test set
LR_y_pred = LR.predict(X_test)

# Evaluate the model
accuracy_LR = accuracy_score(y_test, LR_y_pred)
print("Training (Pre-tuned) Accuracy with LR: {}".format(accuracy_LR))

Training (Pre-tuned) Accuracy with LR: 0.5705521472392638


In [40]:
# hyperparameter with randomized search CV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform

# Define Pipeline
LR_pipeline = Pipeline([
    # Variable Selection and Hyperparameter Tuning
    ('feature_selection', SelectFromModel(LogisticRegression())),
    ('classifier', LogisticRegression())
])

# Define param distributions for pipeline only
LR_pipeline_param_distributions = {
    'classifier__penalty': ['l1', 'l2', 'elasticnet'],
    'classifier__C': uniform(loc=0.01, scale=0.99),  # Uniform distribution between 0.01 and 1
    'classifier__tol': uniform(loc=1e-3, scale=0.099),  # Uniform distribution between 0.001 and 0.1
    'classifier__max_iter': [500, 1000, 5000],  # Fixed choices
    'classifier__solver': ['saga'],
    'classifier__l1_ratio': uniform(loc=0, scale=1)  # Uniform distribution between 0 and 1 for ElasticNet
}

# Perform Randomized Search CV (10: 2 min or 50: 16 mins)
LR_random_search = RandomizedSearchCV(LR_pipeline, LR_pipeline_param_distributions, n_iter=50, cv=5, random_state=42)
LR_random_search.fit(X_val, y_val)

# Retrieve the best estimator after random search CV is fitted
best_LR_estimator = LR_random_search.best_estimator_

# Access the coefficients of the Logistic Regression model
lr_coefficients = best_LR_estimator.named_steps['classifier'].coef_[0]

# Identify the features with the highest absolute coefficients
top_indices = lr_coefficients.argsort()[-10:][::-1]  # Get indices of top 10 features
top_features = [X_val.columns[i] for i in top_indices]  # Get feature names from indices

# Extract the names of the top 10 features
print("Top 10 Most Important Features:")
for feature in top_features:
    print(feature)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear

Top 10 Most Important Features:
popularity
explicit
duration


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(


## On Combined Feature-Engineered Data